# Pedestrian Accessibility of Prague
## Children of age 10-16
### Part 1. Data aquisition and cleaning

First part of this study is focused on data aquisition and cleaning. For our research we collected a number of datasets such as, districts shapes, population, points of interests (schools, educational centers, libraries, sport facilities, playgrounds). Datasets were aquired from different open data sources: 
- District`s shapes: IPR Praha <a>http://www.geoportalpraha.cz</a>
- Population of the administrative districts of Czezch Republic: Czech Statistical Office <a>https://www.czso.cz</a>
- Educational facilities of Prague: The Ministry of Education, Youth and Sports <a>http://www.msmt.cz/</a> 
- Libraries: Wikipedia <a>https://cs.wikipedia.org/wiki/M%C4%9Bstsk%C3%A1_knihovna_v_Praze</a>
- Sport facilities: Opendata Prague <a>http://opendata.praha.eu/about</a>
- Playgrounds: Opendata Prague <a>http://opendata.praha.eu/about</a>

As the result we build to datasetes:
- Children population by districts in Prague <a url='https://github.com/tonnyeremin/Urban-Data-Science/raw/master/prague_district_population.csv'>prague_district_population.csv</a>
- Points of interests  <a url='https://github.com/tonnyeremin/Urban-Data-Science/blob/master/prague_pois.csv'>prague_pois.csv</a>   


### 1. Environment

In [51]:
#!conda install -c conda-forge geocoder #Uncomment this cell to install geocoder package if it is not yet installed

In [52]:
import sys
import io
import pandas as pd
import numpy as np
import json
import urllib.request
import requests
from pandas.io.json import json_normalize

In [53]:
files = {'districts':'prague_district_population.csv', 'poi':'prague_pois.csv'}

In [54]:
# @hidden_cell
storage_creds = {'apikey' : 'rqrlKTO277J6k4N_5X_wpI62WqwslFDxspY7o2Nb6s0A',
                 'iam_serviceid_crn' : 'crn:v1:bluemix:public:cloud-object-storage:global:a/8aa0fa0d4ad4480b8bfdf1c4d79f9442:021a8d33-89af-44aa-b548-e6f14a067d79:bucket:prague-data-set',
                 'auth_ep': 'https://iam.cloud.ibm.com/identity/token',
                 'ep': 'https://s3.private.eu-de.cloud-object-storage.appdomain.cloud',
                 'bucket' : 'prague-data-set'}

In [55]:
#Define storage creds here
# storage_creds = {'apikey' : 'apikey',
#                 'iam_serviceid_crn' : 'iam_serviceid_crn',
#                 'auth_ep': 'https://iam.cloud.ibm.com/identity/token',
#                 'ep': 'https://s3.private.eu-de.cloud-object-storage.appdomain.cloud',
#                 'bucket' : 'bucket'}

In [56]:
#Upload and download functions for IBM data storage

import sys
from ibm_botocore.client import Config
import ibm_boto3

def upload_file(credentials,local_file_name,key): 
    storage = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['apikey'],
    ibm_service_instance_id=credentials['iam_serviceid_crn'],
    ibm_auth_endpoint=credentials['auth_ep'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ep'])
    
    try:
        res=storage.upload_file(Filename=local_file_name, Bucket=credentials['bucket'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File {} Uploaded'.format(local_file_name))
        
def download_file(credentials,local_file_name,key):  
    storage = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['apikey'],
    ibm_service_instance_id=credentials['iam_serviceid_crn'],
    ibm_auth_endpoint=credentials['auth_ep'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ep'])
    try:
        res= storage.download_file(Bucket=credentials['bucket'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File {} Downloaded'.format(local_file_name))

# Coordinates retrieve function
import geocoder

def get_coordinates(dataFrame, index_row):
    dict_coordinates = {}
    total_count = len(dataFrame.index)
    current = 0
    errors = 0
    for index, row in dataFrame.iterrows():
        try:
            g = geocoder.arcgis(row[index_row])
            lat = g.json['lat']
            lng = g.json['lng']
            dict_coordinates[index] = [lat, lng]
            current+=1
        except:
            errors+=1
            print ('Failed to get coordinates for {}: {}'.format(index_row, sys.exc_info()[0]))
    
    dataFrame['latitude'] = 0.0
    dataFrame['longitude'] = 0.0
    
    for k, v in dict_coordinates.items():
        dataFrame.loc[k,'latitude']=v[0]
        dataFrame.loc[k,'longitude']=v[1]
        
    print('Done: Total: {} Success: {} Error {}'.format(total_count, current, errors))
print('Environment was initializied')    

Environment was initializied


### 2. Prague`s districts borders and population

First step is to determine the shape, location and population of the administrative disctrics of Prague.

In [57]:
mestky_casty_url = 'http://opendata.iprpraha.cz/CUR/DTMP/TMMESTSKECASTI_P/WGS_84/TMMESTSKECASTI_P.json'
results = requests.get(mestky_casty_url).json(encoding = "utf8")
mestky_casty = json_normalize(results['features']) 
print('Loaded: {} districts'.format(mestky_casty.shape[0]))

Loaded: 57 districts


In [58]:
url_population =  'https://www.czso.cz/documents/10180/25233177/sldb_zv.csv'
df_population = pd.read_csv(url_population,encoding = "ISO 8859-2")

df_population = df_population[(df_population.uzcis == 44)& (df_population.nazev.str.find('Praha') != -1)][['nazev','u01','u04', 'u05', 'u06']]
df_population.rename(columns={'nazev':'Name','u01':'Total', 'u04':'Kids', 'u05':'Middle', 'u06':'Senior'}, inplace = True)
df_population['Name'] = df_population['Name'].map(lambda x: x.lower())

print('Loaded: {} districts'.format(df_population.shape[0]))

Loaded: 57 districts


Join district geography and population datasets

In [59]:
geo_unique = np.array(mestky_casty['properties.NAZEV_MC'].apply(lambda x: x.lower()).unique())
population_unique = df_population['Name'].unique()

print('Unique districts in geo dataset: {}'.format(len(geo_unique)))
print('Unique districts in population dataset: {}'.format(len(population_unique)))

districts_diff_geo = list(set(geo_unique)-set(population_unique))
districts_diff_pop = list(set(population_unique)-set(geo_unique))

print (districts_diff_geo)

print (districts_diff_pop)

print ('Datasetes difference: {} {}'.format(len(districts_diff_geo), len(districts_diff_pop)))

Unique districts in geo dataset: 57
Unique districts in population dataset: 57
['praha-libuš', 'praha-nebušice', 'praha-štěrboholy', 'praha-šeberov']
['praha-\x8atěrboholy', 'praha-\x8aeberov', 'praha-nebu\x9aice', 'praha-libu\x9a']
Datasetes difference: 4 4


In [60]:
df_population.loc[df_population.Name == 'praha-libu\x9a', 'Name'] = 'praha-libuš'
df_population.loc[df_population.Name == 'praha-\x8aeberov', 'Name'] = 'praha-šeberov'
df_population.loc[df_population.Name == 'praha-nebu\x9aice', 'Name'] = 'praha-nebušice'
df_population.loc[df_population.Name == 'praha-\x8atěrboholy', 'Name'] = 'praha-štěrboholy'

population_unique = df_population['Name'].unique()
districts_diff_pop = list(set(population_unique)-set(geo_unique))

print('Data set equal: {}'.format(len(districts_diff_pop)==0)) 

Data set equal: True


Creating resulting dataset for geographical and demographical data

In [61]:
result = []
#converting from json to pandas data frame
result.append([
    v['properties']['NAZEV_MC'].lower(),
    v['geometry']['coordinates'][0],
    v['properties']['PLOCHA']] for v in results['features'])
    
df_prague_districts = pd.DataFrame([item for result in result for item in result])
df_prague_districts.columns = ['Name', 'Geometry', 'Area']

#Joing datasets
df_prague = df_prague_districts.set_index('Name').join(df_population.set_index('Name'))

quotient = df_prague['Middle']/1000
df_prague['Kids_per_1000'] = df_prague['Kids']/quotient
df_prague.sort_values('Name', inplace = True)
df_prague.reset_index(inplace=True)
df_prague.shape

(57, 8)

In [62]:
get_coordinates(df_prague, 'Name')
df_prague.head()

Done: Total: 57 Success: 57 Error 0


,Name,Geometry,Area,Total,Kids,Middle,Senior,Kids_per_1000,latitude,longitude
0,praha 1,"[[14.410891049000043, 50.078674687000046], [14...",5538443.86,30561.0,2391.0,22963.0,4594.0,104.124026,50.08728,14.41742
1,praha 10,"[[14.531321086000048, 50.072240288000046], [14...",18599366.98,113200.0,12213.0,76625.0,23937.0,159.386623,50.06762,14.46016
2,praha 11,"[[14.54355294800007, 50.03618763800006], [14.5...",9793679.84,75741.0,8688.0,54983.0,11816.0,158.012477,50.03178,14.50719
3,praha 12,"[[14.450632163000023, 50.01452735600003], [14....",23317909.06,53515.0,6156.0,39699.0,7480.0,155.066878,50.00564,14.40462
4,praha 13,"[[14.320621949000042, 50.04010680700003], [14....",13196802.19,59906.0,7985.0,46514.0,5109.0,171.668745,50.05163,14.34231


In [63]:
file_name = files['districts']

df_prague.to_csv(file_name)
upload_file(storage_creds,file_name,file_name)

File prague_district_population.csv Uploaded


### 3. Points of interests
### 3.1 Schools and other educational facilities

In [64]:
url_schools = 'https://rejstriky.msmt.cz/opendata/vrejcz010.xml'
file_schools = 'schools.xml'
results = requests.get(url_schools)
results.content
with open(file_schools, 'w') as file:
    file.write(results.text)
print('Loaded') 

Loaded


In [65]:
#Converting XML to pandas dataframe 

import xml.etree.ElementTree as et 
xtree = et.parse(file_schools)
xroot = xtree.getroot()

dic_scools = []

try:
    for entry in xroot.findall('PravniSubjekt'):
        place_group = entry.find('SkolyZarizeni')
        if(place_group is None):
            continue
        for place in place_group.findall('SkolaZarizeni'):
            s_id = place.find('IZO').text
            s_type = place.find('SkolaDruhTyp').text
            s_name = place.find('SkolaPlnyNazev').text
            s_capasity = place.find('SkolaKapacita').text
            s_adress = place.find('SkolaMistaVykonuCinnosti')
            s_actual_add = s_adress.find('SkolaMistoVykonuCinnosti')
            s_addres1 =  s_actual_add.find('MistoAdresa1').text
            s_addres2 =  s_actual_add.find('MistoAdresa2').text
            s_addres3 =  s_actual_add.find('MistoAdresa3').text
            dic_scools.append([s_id, s_name,  s_type, s_capasity, '{} {} {}'.format(s_addres1, s_addres2, s_addres3)])
    print('Completed. Total schools and educational centers count: {}'.format(len(dic_scools)))
except:
    print ('Exception', sys.exc_info()[0]) 
    
columns = ['id', 'name', 'type', 'capacity', 'address']
df_education = pd.DataFrame(dic_scools, columns = columns)

print('Dataframe created: {},{}'.format(df_education.shape[0], df_education.shape[1]))

#upload to datastore
df_prague.to_csv('prague_schools.csv')
upload_file(storage_creds,'prague_schools.csv','prague_schools.csv')

Completed. Total schools and educational centers count: 2273
Dataframe created: 2273,5
File prague_schools.csv Uploaded


In [66]:
#Check for predefinied types at schools dataframe
types = df_education['type'].unique()
print('Types in XML file')
for t in types:
    print(t,df_education[df_education.type == t].iloc[0,1])

#filtering types
with pd.option_context('mode.chained_assignment', None):
    types = ['B00', 'F10', 'C00','H22', 'G11']
    types_shu = types[0:3]
    df_education_selected = df_education.loc[df_education.type.isin(types)]
    df_education_selected.loc[df_education_selected['type'].isin(types_shu), 'Type'] = 'school'
    df_education_selected = df_education_selected.fillna('educatioanal center')

print('Schools and educational centers count {}'.format(df_education_selected.shape[0]))
print('Unique types {}'.format(df_education_selected['Type'].unique()))

Types in XML file
A00 Mateřská škola
L11 Školní jídelna
L13 Školní jídelna - výdejna
B00 Základní škola
M60 Přípravný stupeň základní školy speciální
G21 Školní družina
G22 Školní klub
F10 Základní umělecká škola
C00 Střední škola
D00 Konzervatoř
M20 Školní knihovna
E00 Vyšší odborná škola
M79 Jiné účelové zařízení
H22 Domov mládeže
G11 Dům dětí a mládeže
G40 Zařízení pro další vzdělávání pedagogických pracovníků
M40 Středisko praktického vyučování
H21 Internát
K20 Speciálně pedagogické centrum
G12 Stanice zájmových činností
K10 Pedagogicko-psychologická poradna
F20 Jazyková škola s právem státní jazykové zkoušky
J12 Dětský domov se školou
J21 Středisko výchovné péče
J14 Diagnostický ústav
J11 Dětský domov
J13 Výchovný ústav
L12 Školní jídelna - vývařovna
H10 Škola v přírodě
A15 Mateřská škola (lesní mateřská škola)
Schools and educational centers count 560
Unique types ['school' 'educatioanal center']


In [67]:
#Cleaning and retriving coordinates
df_education_selected.loc[0:, 'District_Name'] = df_education_selected.loc[0:,'address'].apply(lambda x: ' '.join(x.split()[-2:]).lower())

columns_to_drop = ['id','name','capacity', 'type']
df_education_selected.drop(columns = columns_to_drop, inplace = True)

get_coordinates(df_education_selected, 'address')

df_education_selected.drop(columns = ['address'], inplace= True)
df_education_selected.head()

Done: Total: 560 Success: 560 Error 0


,Type,District_Name,latitude,longitude
5,school,praha 4,50.008620,14.448992
22,school,praha 1,50.080344,14.415264
25,school,praha 1,50.090834,14.428853
26,school,praha 1,50.090834,14.428853
30,school,praha 4,50.001684,14.413886


In [68]:
df_prague_poi = pd.DataFrame(df_education_selected)

poi_file_name = files['poi']
df_prague_poi.to_csv(poi_file_name)
upload_file(storage_creds,poi_file_name,poi_file_name)

File prague_pois.csv Uploaded


### 3.2 Libraries

In [71]:
url_libs = 'https://cs.wikipedia.org/wiki/M%C4%9Bstsk%C3%A1_knihovna_v_Praze'

f = urllib.request.urlopen(url_libs)
html = f.read()

try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup

parsed_html = BeautifulSoup(html)
tag_header = parsed_html.find_all('h4')
district_tags = []

for tag in tag_header:
    tag_match = False
    district =''
    for child in tag.children:
        if child.get("class")[0] == 'mw-headline':
            district = child.get_text().lower()
            tag_match = True
    if tag_match == True:
        nextsibling = tag.next_sibling
        while  True:
            if nextsibling.find('ul') != -1 :
                lists = nextsibling.find_all('li')
                for lib in  lists:
                    district_tags.append(['library',district,lib.get_text()])
                break
            else:
                nextsibling = nextsibling.next_sibling

df_libs = pd.DataFrame(data=district_tags)
df_libs.columns = ['Type', 'District_Name', 'Address']
print('Libraries count: {}'.format(df_libs.shape[0]))

get_coordinates(df_libs, 'Address')
df_libs.drop(columns=['Address'], inplace = True)
df_libs.head()

Libraries count: 41
Done: Total: 41 Success: 41 Error 0


,Type,District_Name,latitude,longitude
0,library,praha 1,50.079501,14.424045
1,library,praha 1,50.087778,14.389993
2,library,praha 2,50.071910,14.436970
3,library,praha 2,50.073620,14.416410
4,library,praha 2,50.065790,14.424680


In [72]:
df_prague_poi = pd.concat([df_prague_poi,df_libs], sort=True)

poi_file_name = files['poi']
df_prague_poi.to_csv(poi_file_name)
upload_file(storage_creds,poi_file_name,poi_file_name)

File prague_pois.csv Uploaded


### 3.3 Sport centers and clubs

In [73]:
url_sport = 'http://opendata.praha.eu/datastore/dump/5d1ee13f-f6e9-4ee9-a1bd-48d5ca2bb867?format=json'
results = requests.get(url_sport).json(encoding = "utf8")
result = []

result.append([
    'sport',
    'praha {}'.format(v[6]),
     v[2]] for v in results['records'])
    
df_sport = pd.DataFrame([item for result in result for item in result])
df_sport.columns = ['Type', 'District_Name', 'Address']

print('Sport centers and clubs count: {}'.format(df_sport.shape[0]))

get_coordinates(df_sport, 'Address')

df_sport.drop(columns=['Address'], inplace = True)

df_sport.head()

Sport centers and clubs count: 877
Done: Total: 877 Success: 877 Error 0


,Type,District_Name,latitude,longitude
0,sport,praha 5,50.052154,14.360772
1,sport,praha 5,50.068925,14.345478
2,sport,praha 1,50.085924,14.431106
3,sport,praha 12,50.009011,14.447198
4,sport,praha 7,50.096690,14.440140


In [74]:
df_prague_poi = pd.concat([df_prague_poi, df_sport], sort=True)

poi_file_name = files['poi']
df_prague_poi.to_csv(poi_file_name)
upload_file(storage_creds,poi_file_name,poi_file_name)

File prague_pois.csv Uploaded


### 3.4 Outdoor playgrounds

In [75]:
url_playgrounds = 'http://opendata.praha.eu/dataset/3c3ca9ca-fbc0-4f97-b624-ed967f5d9a24/resource/e19c2e29-5e33-4449-8847-5dc8f5b8a2f2/download/db144c03-1a0f-456f-a32b-9c48ccfc0813-playgrounds.json'
results = requests.get(url_playgrounds).json(encoding = "utf8")

print('Downloaded')

poi_type = 'playground'
result = []

result.append([
    poi_type,
    v['properties']['district'].lower(),
    v['geometry']['coordinates'][0],
    v['geometry']['coordinates'][1]] for v in results['features'])
    
df_playgrounds = pd.DataFrame([item for result in result for item in result])
df_playgrounds.columns = ['Type', 'District_Name', 'latitude','longitude']

indeces=df_playgrounds.loc[df_playgrounds.District_Name.str.contains('[-][0-9]', regex=True)].index
df_playgrounds.loc[indeces, 'District_Name'] = df_playgrounds.loc[indeces, 'District_Name'].str.replace('-', ' ')
print('Outdoor playgrounds count: {}'.format(df_playgrounds.shape[0]))

df_playgrounds.head()

Downloaded
Outdoor playgrounds count: 145


,Type,District_Name,latitude,longitude
0,playground,praha-petrovice,14.563237,50.038025
1,playground,praha 8,14.438850,50.134014
2,playground,praha 3,14.476411,50.094387
3,playground,praha 22,14.593131,50.036453
4,playground,praha 15,14.539891,50.043732


In [76]:
df_prague_poi = pd.concat([df_prague_poi, df_playgrounds], sort=True)

poi_file_name = files['poi']
df_prague_poi.to_csv(poi_file_name)
upload_file(storage_creds,poi_file_name,poi_file_name)

File prague_pois.csv Uploaded


In [77]:
df_prague_poi.shape

(1623, 4)

### 4. Results

### 4.1 Children population by districts in Prague. 

In [78]:
print("Total districts: ", len(df_prague))

Total districts:  57


In [79]:
df_prague.head()

,Name,Geometry,Area,Total,Kids,Middle,Senior,Kids_per_1000,latitude,longitude
0,praha 1,"[[14.410891049000043, 50.078674687000046], [14...",5538443.86,30561.0,2391.0,22963.0,4594.0,104.124026,50.08728,14.41742
1,praha 10,"[[14.531321086000048, 50.072240288000046], [14...",18599366.98,113200.0,12213.0,76625.0,23937.0,159.386623,50.06762,14.46016
2,praha 11,"[[14.54355294800007, 50.03618763800006], [14.5...",9793679.84,75741.0,8688.0,54983.0,11816.0,158.012477,50.03178,14.50719
3,praha 12,"[[14.450632163000023, 50.01452735600003], [14....",23317909.06,53515.0,6156.0,39699.0,7480.0,155.066878,50.00564,14.40462
4,praha 13,"[[14.320621949000042, 50.04010680700003], [14....",13196802.19,59906.0,7985.0,46514.0,5109.0,171.668745,50.05163,14.34231


### 4.2 Points of intererests

In [80]:
print("Points of interest colleted for ", len(df_prague_poi['District_Name'].unique()), ' districts')
print("Total points of interest: ", len(df_prague_poi))
print("Total points of interest types: ", len(df_prague_poi['Type'].unique()))
for t in df_prague_poi['Type'].unique():
    print("   {}: {}".format(t, len(df_prague_poi[df_prague_poi.Type == t])))

Points of interest colleted for  36  districts
Total points of interest:  1623
Total points of interest types:  5
   school: 517
   educatioanal center: 43
   library: 41
   sport: 877
   playground: 145


In [81]:
df_prague_poi.head()

,District_Name,Type,latitude,longitude
5,praha 4,school,50.008620,14.448992
22,praha 1,school,50.080344,14.415264
25,praha 1,school,50.090834,14.428853
26,praha 1,school,50.090834,14.428853
30,praha 4,school,50.001684,14.413886
